In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
import torch
import os
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset
from dataclasses import dataclass
from typing import Optional, List
from fuzzywuzzy import fuzz
import evaluate

In [3]:
def is_approximate_substring(substring, string, threshold=70):
    for i in range(len(string) - len(substring) + 1):
        window = string[i:i+len(substring)]
        similarity_ratio = fuzz.ratio(substring, window)
        if similarity_ratio >= threshold:
            return True
    return False

In [4]:
conversation_training_path = "data/dataset_v2/training"

user_id = []
previous_interactions = []
recommended_app_name = []
turns = []
recommend_indexes = []


# List all files in the folder
files = os.listdir(conversation_training_path)

# Loop through each file in the directory
for filename in tqdm(files):
    # Construct the full file path
    file_path = os.path.join(conversation_training_path, filename)

    # Check if the current item is a file
    if os.path.isfile(file_path):
        # Open and read the file
        with open(file_path, 'r') as file:
            # Read the content of the file
            file_content = file.read().lower()
            
            # finding the user id
            index_1 = file_content.find("user's previous interactions")
            user_id.append(file_content[9:index_1].rstrip('\n'))
            
            # finding the User's Previous Interactions
            index_2 = file_content.find("recommended app name:")
            previous_interactions_arr = file_content[index_1+29:index_2].rstrip('\n').split("app name:")
            previous_interactions_arr_filtered = []
            for previous_interaction in previous_interactions_arr[1:]:
                previous_interactions_arr_filtered.append(previous_interaction[:previous_interaction.find(" | ")])
            if len(previous_interactions_arr_filtered) > 0:
                previous_interactions.append(",".join(previous_interactions_arr_filtered))
            else:
                previous_interactions.append(None)
            
            # finding recommended app name
            index_3 = file_content[index_2:].find("package name")
            recommended = file_content[index_2+22:index_2+index_3-3].rstrip('\n')
            recommended_app_name.append(recommended)
            
            # finding each turns
            dialog_turns = []
            dialog_index = 0
            COMPUTER_index = file_content.find("computer:")
            file_content = file_content[COMPUTER_index:]
            found_recommender = False
            while True:
                HUMAN_index = file_content.find("human:")
                if HUMAN_index == -1:
                    break
                turn = file_content[:HUMAN_index].rstrip('\n') # computer dialog
                if (recommended in turn) and not found_recommender:
                    recommend_indexes.append(dialog_index)
                    found_recommender = True
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[HUMAN_index:]
                
                COMPUTER_index = file_content.find("computer:")
                turn = file_content[:COMPUTER_index].rstrip('\n') # human dialog
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[COMPUTER_index:]
                
            if not found_recommender: # approximately finding the recommender turn
                for i, dialog_turn in enumerate(dialog_turns):
                    if is_approximate_substring(recommended, dialog_turn):
                        recommend_indexes.append(i)
                        found_recommender = True
                        break
                    
            if not found_recommender:
                recommend_indexes.append(-1)
                        
            turns.append(dialog_turns)

print(len(user_id))
print(len(previous_interactions))
print(len(recommended_app_name))
print(len(recommend_indexes))
df_recommender_train = pd.DataFrame({"user_id": user_id, "previous_interactions":previous_interactions, "recommended_app_name":recommended_app_name, "turns": turns, "recommend_indexes":recommend_indexes})
print(f"\nnumber of rows: {len(df_recommender_train)}")

  0%|          | 26/8720 [00:00<00:34, 249.81it/s]

100%|██████████| 8720/8720 [00:25<00:00, 347.46it/s]

8720
8720
8720
8720

number of rows: 8720


In [5]:
df_recommender_train = df_recommender_train[(df_recommender_train["recommend_indexes"] != -1) & (df_recommender_train["turns"].apply(lambda x: len(x) > 0))]

In [6]:
conversation_validation_path = "data/dataset_v2/validation"

user_id = []
previous_interactions = []
recommended_app_name = []
turns = []
recommend_indexes = []


# List all files in the folder
files = os.listdir(conversation_validation_path)

# Loop through each file in the directory
for filename in tqdm(files):
    # Construct the full file path
    file_path = os.path.join(conversation_validation_path, filename)

    # Check if the current item is a file
    if os.path.isfile(file_path):
        # Open and read the file
        with open(file_path, 'r') as file:
            # Read the content of the file
            file_content = file.read().lower()
            
            # finding the user id
            index_1 = file_content.find("user's previous interactions")
            user_id.append(file_content[9:index_1].rstrip('\n'))
            
            # finding the User's Previous Interactions
            index_2 = file_content.find("recommended app name:")
            previous_interactions_arr = file_content[index_1+29:index_2].rstrip('\n').split("app name:")
            previous_interactions_arr_filtered = []
            for previous_interaction in previous_interactions_arr[1:]:
                previous_interactions_arr_filtered.append(previous_interaction[:previous_interaction.find(" | ")])
            if len(previous_interactions_arr_filtered) > 0:
                previous_interactions.append(",".join(previous_interactions_arr_filtered))
            else:
                previous_interactions.append(None)
            
            # finding recommended app name
            index_3 = file_content[index_2:].find("package name")
            recommended = file_content[index_2+22:index_2+index_3-3].rstrip('\n')
            recommended_app_name.append(recommended)
            
            # finding each turns
            dialog_turns = []
            dialog_index = 0
            COMPUTER_index = file_content.find("computer:")
            file_content = file_content[COMPUTER_index:]
            found_recommender = False
            while True:
                HUMAN_index = file_content.find("human:")
                if HUMAN_index == -1:
                    break
                turn = file_content[:HUMAN_index].rstrip('\n') # computer dialog
                if (recommended in turn) and not found_recommender:
                    recommend_indexes.append(dialog_index)
                    found_recommender = True
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[HUMAN_index:]
                
                COMPUTER_index = file_content.find("computer:")
                turn = file_content[:COMPUTER_index].rstrip('\n') # human dialog
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[COMPUTER_index:]
                
            if not found_recommender: # approximately finding the recommender turn
                for i, dialog_turn in enumerate(dialog_turns):
                    if is_approximate_substring(recommended, dialog_turn):
                        recommend_indexes.append(i)
                        found_recommender = True
                        break
                    
            if not found_recommender:
                recommend_indexes.append(-1)
                        
            turns.append(dialog_turns)

print(len(user_id))
print(len(previous_interactions))
print(len(recommended_app_name))
print(len(recommend_indexes))
df_recommender_validation = pd.DataFrame({"user_id": user_id, "previous_interactions":previous_interactions, "recommended_app_name":recommended_app_name, "turns": turns, "recommend_indexes":recommend_indexes})
print(f"\nnumber of rows: {len(df_recommender_validation)}")

100%|██████████| 1285/1285 [00:04<00:00, 302.96it/s]

1285
1285
1285
1285

number of rows: 1285


In [7]:
df_recommender_validation = df_recommender_validation[(df_recommender_validation["recommend_indexes"] != -1) & (df_recommender_validation["turns"].apply(lambda x: len(x) > 0))]

In [8]:
count = 0
for turn in df_recommender_train['turns']:
    if len(turn) == 0:
        count += 1
print(count)

count = 0
for recommend_index in df_recommender_train['recommend_indexes']:
    if recommend_index == -1:
        count += 1
print(count)

0
0


In [9]:
count = 0
for turn in df_recommender_validation['turns']:
    if len(turn) == 0:
        count += 1
print(count)

count = 0
for recommend_index in df_recommender_validation['recommend_indexes']:
    if recommend_index == -1:
        count += 1
print(count)

0
0


In [10]:
df_recommender_validation

,user_id,previous_interactions,recommended_app_name,turns,recommend_indexes
0,qwttxhjcryqhatgb,None,wattpad - read & write stories,[computer: hi there! how can i help you today?...,18
1,tugk24mlf1qvjzba,flight sim 2018,"paytm -upi, money transfer, recharge, bill pay...",[computer: hi there! what brings you here toda...,24
2,hjbzfrhcuq2pxoma,"dealdash - bid & save auctions, heart's medic...",wood block puzzle - block game,[computer: hey there! how can i help you today...,20
3,0zm4n7djyrppfrgx,"dangerous fellows: otome game, painnt - pro a...",dynamons 2,[computer: hey there! what are you looking for...,24
4,2c7ofjyraazmwgr3,"bumble - dating. friends. bizz, music player ...",2 player games : the challenge,"[computer: hello! how can i help you today?, h...",20
...,...,...,...,...,...
1280,chyzv6ycokrxisoo,"vector, command & conquer: rivalsãâ¢ãâãâ¢...",my talking hank,"[computer: hello! how can i help you today?, h...",22
1281,pfin90inkk8rauwn,"wattpad beta, soccer star 22 top leagues, rai...",pinterest,[computer: hi there! how can i help you today?...,20
1282,rog58g9ofnggpobe,"journey: diary, journal, my ooredoo myanmar, ...",legoãâ¯ãâ¿ãâ½ friends: heartlake rush,[computer: hey there! what can i help you find...,24
1283,5s1a0sasi88h3mcq,"age of war 2, noblemen: 1896, groupon ãâ¢ãâ...",human anatomy atlas 2021: complete 3d human body,[computer: hi there! what are you looking for ...,12


In [11]:
df_recommender_train

,user_id,previous_interactions,recommended_app_name,turns,recommend_indexes
0,d1o00gcfxa7tj9fc,"snipers vs thieves, retro bowl, be the king: ...",weather forecast - accurate local weather & wi...,"[computer: hello! how can i assist you today?,...",24
1,tjfy73vbqa5acgac,"daily shopping stories, truck simulator 2018 ...",bridge race,[computer: hi there! what are you looking for ...,18
2,okqwykkowhwv8ppo,"shop titans: craft & build, dragon ball z dok...",video editor & maker videoshow,[computer: are you looking for apps that are e...,16
3,och0j9udd8rwbezp,"pepi wonder world: magic isle!, the simsãâ¯ã...",ludo star,"[computer: hi! how can i help you today?, huma...",24
4,86ckfhrgr3my1asp,microsoft onenote: save ideas and organize no...,xender - share music transfer,[computer: hey there! what can i help you with...,22
...,...,...,...,...,...
8715,dkfqslrdxlhbegdr,None,true fear: forsaken souls 1,[computer: hi there! how can i help you today?...,24
8716,8lw0qo1crrzsoef9,"macy's, play to win: win real money, opera ne...",hill climb racing 2,[computer: hey there! how can i help you today...,16
8717,xqbiugex5rwctlsp,"jurassic world alive, google classroom",zombie catchers ãâ¯ãâ¿ãâ½ love to hunt,[computer: sure thing! what kinds of zombie hu...,22
8718,xpo2xk6chiiygdp3,"playmobil luxury mansion, into the dead 2, sc...",township,[computer: hi there! how can i help you today?...,20


In [ ]:
model_checkpoint = "gpt2"
bos = '<|startoftext|>'
eos = '<|endoftext|>'
pad = '<|pad|>'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = GPT2Tokenizer.from_pretrained(model_checkpoint, bos_token=bos, eos_token=eos, pad_token=pad, additional_special_tokens=["computer:", "human:", "previous_interactions:"])

model = GPT2LMHeadModel.from_pretrained(model_checkpoint).to(device)
model.resize_token_embeddings(len(tokenizer))
model_max_length=1024

In [13]:
tokenizer.truncation_side

'right'

In [14]:
@dataclass
class RecommenderItem:
    prompt: str
    generation: Optional[str] = None
    
class recommenderDataset(Dataset):
    def __init__(self, data: List[RecommenderItem]):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx) -> RecommenderItem:
        return self.data[idx]

In [15]:
items_validation = []
for _, row in df_recommender_validation.iterrows():
    if row["previous_interactions"] is not None:
        prompt = bos + "previous_interactions:" + row["previous_interactions"] + "\n"
    else:
        prompt = bos + "previous_interactions: No previous interactions" + "\n"
    for index, turn in enumerate(row["turns"]):
        if index < row["recommend_indexes"]:
            prompt += turn + "\n"
        elif index == row["recommend_indexes"]:
            prompt += "computer: I would recommend the "
            items_validation.append(RecommenderItem(prompt, row["recommended_app_name"] + " app." + eos))
            break
        else:
            print("error!!")

In [16]:
items_train = []
for _, row in df_recommender_train.iterrows():
    if row["previous_interactions"] is not None:
        prompt = bos + "previous_interactions:" + row["previous_interactions"] + "\n"
    else:
        prompt = bos + "previous_interactions: No previous interactions" + "\n"
    for index, turn in enumerate(row["turns"]):
        if index < row["recommend_indexes"]:
            prompt += turn + "\n"
        elif index == row["recommend_indexes"]:
            prompt += "computer: I would recommend the "
            items_train.append(RecommenderItem(prompt, row["recommended_app_name"] + " app." + eos))
            break
        else:
            print("error!!")

In [17]:
items_train[0].prompt

"<|startoftext|>previous_interactions: snipers vs thieves, retro bowl, be the king: judge destiny, x-plane flight simulator, karaoke - sing songs, filemaster: file manage, file transfer power clean, calm - meditate, sleep, relax, dragon raja, funimate video editor & maker, facemoji emoji keyboard&fonts\ncomputer: hello! how can i assist you today?\nhuman: i want to find a weather forecast app for my android phone.\ncomputer: what kind of weather forecast app are you looking for?\nhuman: i want one that is accurate.\ncomputer: do you prefer apps with a rating of everyone?\nhuman: yes, i prefer everyone rated apps.\ncomputer: are you looking for a free app or are you willing to pay?\nhuman: i would prefer a free app.\ncomputer: would you be okay with occasional ads in the app?\nhuman: yes, i can consider apps with ads.\ncomputer: do you prefer apps with high average ratings from users?\nhuman: yes, i prefer apps with at least a 4.8 star rating.\ncomputer: based on your preferences, i rec

In [18]:
items_train[0].generation

'weather forecast - accurate local weather & widget app.<|endoftext|>'

In [19]:
def training_collator(batch: list[recommenderDataset]): # for training a language model
    input_ids = []
    attention_masks = []
    labels = []
    for item in batch:
        prompt_tokens = tokenizer.encode(item.prompt, return_tensors="pt")[0] 
        generation_tokens = tokenizer.encode(item.generation, return_tensors="pt")[0]
        prompt_len = len(prompt_tokens)
        generation_len = len(generation_tokens)
        unused_len = model_max_length - prompt_len - generation_len
        # handling case when input is greater than tokenizer length.
        if unused_len < 0:
            prompt_start_tokens = prompt_tokens[:1]
            trimmed_prompt = prompt_tokens[unused_len * -1 + 1 :] # TODO: you could delete the prompt to reach the first |beginuser| token
            prompt_tokens = torch.cat(
                [prompt_start_tokens, trimmed_prompt], axis=0
            )
            prompt_len = len(prompt_tokens)
            unused_len = 0
        pad = torch.full([unused_len], tokenizer.pad_token_id)
        input_tokens = torch.cat(
            [prompt_tokens, generation_tokens, pad]
        )
        label = torch.cat(
            [
                torch.full(
                    [prompt_len],
                    -100,
                ),
                generation_tokens,
                torch.full([unused_len], -100),
            ]
        )
        attention_mask = torch.cat(
            [
                torch.full([prompt_len + generation_len], 1),
                torch.full([unused_len], 0),
            ]
        )
        input_ids.append(input_tokens)
        attention_masks.append(attention_mask)
        labels.append(label)

    out = {
        "input_ids": torch.stack(input_ids),
        "attention_mask": torch.stack(attention_masks),
        "labels": torch.stack(labels),
    }

    return out

In [20]:
training_args = TrainingArguments(
    output_dir="models/GPT2_previous_interactions",
    num_train_epochs=5,
    # logging_steps=500,
    # logging_dir=self.cfg.logging_dir,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_strategy="steps",
    evaluation_strategy="steps",
    save_steps=0.3,#self.cfg.save_steps,
    eval_steps=0.3, #self.cfg.eval_steps,
    save_total_limit=3,
    gradient_accumulation_steps=3, #gradient_accumulation_steps,
    per_device_train_batch_size=4, #train_batch_size,
    per_device_eval_batch_size=4, #self.cfg.eval_batch_size,
    warmup_steps=100,
    weight_decay=0.01,
    # dataloader_drop_last=True,
    disable_tqdm=False,
    push_to_hub=False
)


trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=recommenderDataset(items_train),
        eval_dataset=recommenderDataset(items_validation), #dm.datasets[DataNames.dev_language_model.value],
        data_collator=training_collator,
    )

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [21]:
trainer.train()
trainer.save_model()

Step,Training Loss,Validation Loss


In [ ]:
torch.cuda.empty_cache()

# Load the model and test it on the test dataset

In [3]:
def is_approximate_substring(substring, string, threshold=70):
    for i in range(len(string) - len(substring) + 1):
        window = string[i:i+len(substring)]
        similarity_ratio = fuzz.ratio(substring, window)
        if similarity_ratio >= threshold:
            return True
    return False

In [4]:
conversation_test_path = "data/dataset_v2/testing"

user_id = []
previous_interactions = []
recommended_app_name = []
turns = []
recommend_indexes = []


# List all files in the folder
files = os.listdir(conversation_test_path)

# Loop through each file in the directory
for filename in tqdm(files):
    # Construct the full file path
    file_path = os.path.join(conversation_test_path, filename)

    # Check if the current item is a file
    if os.path.isfile(file_path):
        # Open and read the file
        with open(file_path, 'r') as file:
            # Read the content of the file
            file_content = file.read().lower()
            
            # finding the user id
            index_1 = file_content.find("user's previous interactions")
            user_id.append(file_content[9:index_1].rstrip('\n'))
            
            # finding the User's Previous Interactions
            index_2 = file_content.find("recommended app name:")
            previous_interactions_arr = file_content[index_1+29:index_2].rstrip('\n').split("app name:")
            previous_interactions_arr_filtered = []
            for previous_interaction in previous_interactions_arr[1:]:
                previous_interactions_arr_filtered.append(previous_interaction[:previous_interaction.find(" | ")])
            if len(previous_interactions_arr_filtered) > 0:
                previous_interactions.append(",".join(previous_interactions_arr_filtered))
            else:
                previous_interactions.append(None)
            
            # finding recommended app name
            index_3 = file_content[index_2:].find("package name")
            recommended = file_content[index_2+22:index_2+index_3-3].rstrip('\n')
            recommended_app_name.append(recommended)
            
            # finding each turns
            dialog_turns = []
            dialog_index = 0
            COMPUTER_index = file_content.find("computer:")
            file_content = file_content[COMPUTER_index:]
            found_recommender = False
            while True:
                HUMAN_index = file_content.find("human:")
                if HUMAN_index == -1:
                    break
                turn = file_content[:HUMAN_index].rstrip('\n') # computer dialog
                if (recommended in turn) and not found_recommender:
                    recommend_indexes.append(dialog_index)
                    found_recommender = True
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[HUMAN_index:]
                
                COMPUTER_index = file_content.find("computer:")
                turn = file_content[:COMPUTER_index].rstrip('\n') # human dialog
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[COMPUTER_index:]
                
            if not found_recommender: # approximately finding the recommender turn
                for i, dialog_turn in enumerate(dialog_turns):
                    if is_approximate_substring(recommended, dialog_turn):
                        recommend_indexes.append(i)
                        found_recommender = True
                        break
                    
            if not found_recommender:
                recommend_indexes.append(-1)
                        
            turns.append(dialog_turns)

print(len(user_id))
print(len(previous_interactions))
print(len(recommended_app_name))
print(len(recommend_indexes))
df_recommender_test = pd.DataFrame({"user_id": user_id, "previous_interactions":previous_interactions, "recommended_app_name":recommended_app_name, "turns": turns, "recommend_indexes":recommend_indexes})
print(f"\nnumber of rows: {len(df_recommender_test)}")

100%|██████████| 2557/2557 [00:07<00:00, 344.37it/s]

2557
2557
2557
2557

number of rows: 2557


In [5]:
df_recommender_test = df_recommender_test[(df_recommender_test["recommend_indexes"] != -1) & (df_recommender_test["turns"].apply(lambda x: len(x) > 0))]

In [6]:
@dataclass
class RecommenderItem:
    prompt: str
    generation: Optional[str] = None
    
class recommenderDataset(Dataset):
    def __init__(self, data: List[RecommenderItem]):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx) -> RecommenderItem:
        return self.data[idx]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bos = '<|startoftext|>'
eos = '<|endoftext|>'
pad = '<|pad|>'

tokenizer = GPT2Tokenizer.from_pretrained("gpt2", bos_token=bos, eos_token=eos, pad_token=pad, additional_special_tokens=["computer:", "human:", "previous_interactions:"], padding_side='left')

model = GPT2LMHeadModel.from_pretrained("models/GPT2_previous_interactions").to(device)
model.resize_token_embeddings(len(tokenizer))

In [8]:
items_test = []
for _, row in df_recommender_test.iterrows():
    if row["previous_interactions"] is not None:
        prompt = bos + "previous_interactions:" + row["previous_interactions"] + "\n"
    else:
        prompt = bos + "previous_interactions: No previous interactions" + "\n"
    for index, turn in enumerate(row["turns"]):
        if index < row["recommend_indexes"]:
            prompt += turn + "\n"
        elif index == row["recommend_indexes"]:
            prompt += "computer: I would recommend the "
            items_test.append(RecommenderItem(prompt, row["recommended_app_name"] + " app."))
            break
        else:
            print("error!!")

In [9]:
bleu = evaluate.load("bleu")

In [10]:
def chunk(list_of_elements, batch_size): # using this chunk function, we can split our data to multiple batches
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]

def evaluate_recommender(dataset, model, tokenizer, batch_size=8, device=device, threshold=70):
  prompt_arr = [data.prompt for data in dataset]
  generation_arr = [data.generation for data in dataset]
  prompt_batches = list(chunk(prompt_arr, batch_size))
  generation_batches = list(chunk(generation_arr, batch_size))
  max_length=992
  generation_length = 32
  correctly_predicted = []
  for prompt_batch, generation_batch in tqdm(zip(prompt_batches, generation_batches), total = len(generation_batches)):

    inputs = tokenizer(prompt_batch, max_length=max_length, truncation=True, padding="max_length", return_tensors="pt") 

    generations_predicted = model.generate(input_ids=inputs["input_ids"].to(device), attention_mask=inputs["attention_mask"].to(device),
                            max_new_tokens=generation_length,
                            num_beams=8,
                            eos_token_id=tokenizer.eos_token_id,
                            pad_token_id=tokenizer.pad_token_id,
                            bos_token_id=tokenizer.bos_token_id) # length_penalty=0.8, Set length_penalty to values < 1.0 in order to encourage the model to generate shorter sequences, to a value > 1.0 in order to encourage the model to produce longer sequences.

    generations_predicted = generations_predicted[:, max_length:] # we only need the generation part, not the prompt part.
    decoded_generations = [tokenizer.decode(generation, skip_special_tokens=True, clean_up_tokenization_spaces=True).replace(" app.", "")  for generation in generations_predicted]
    generation_batch = [generation.replace(" app.", "") for generation in generation_batch]
    
    correctly_predicted.extend([1 if fuzz.ratio(predicted, ground_truth) > threshold else 0 for predicted, ground_truth in zip(decoded_generations, generation_batch)])


  return correctly_predicted

In [12]:
correctly_predicted = evaluate_recommender(recommenderDataset(items_test), model, tokenizer, batch_size=4, device=device, threshold=95)
success_rate = sum(correctly_predicted) / len(correctly_predicted)
print("success_rate: ", success_rate)

100%|██████████| 634/634 [13:30<00:00,  1.28s/it]

success_rate:  0.666403785488959


: 

In [11]:
correctly_predicted = evaluate_recommender(recommenderDataset(items_test), model, tokenizer, batch_size=4, device=device)
success_rate = sum(correctly_predicted) / len(correctly_predicted)
print("success_rate: ", success_rate)

  0%|          | 0/634 [00:00<?, ?it/s]

100%|██████████| 634/634 [13:30<00:00,  1.28s/it]

success_rate:  0.705441640378549
